In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os

In [26]:
def abrirImagenesEscaladas( carpeta, escala=32 ):
    # abre todas las imagenes de la carpeta, y las escala de tal forma que midan (escala x escala)px
    # devuelve las imagenes aplanadas -> vectores de tamano escala^2 con valores entre 0 y 1
    imagenes = []

    for dirpath, dirnames, filenames in os.walk(carpeta):
        for file in filenames:
            img = Image.open( os.path.join(carpeta, file) )
            img = img.resize((escala, escala))
            img.convert('1')
            img = np.asarray(img)
            if len(img.shape)==3:
                img = img[:,:,0].reshape((escala**2 )) / 255
            else:
                img = img.reshape((escala**2 )) / 255
            
            imagenes.append( img )

    return imagenes

In [ ]:
# Data
img_train_normal = abrirImagenesEscaladas('./chest_xray/train/NORMAL/')
# img_train_neumonia = abrirImagenesEscaladas('./chest_xray/train/PNEUMONIA/') # NO FUNCIONA :(
img_test_normal = abrirImagenesEscaladas('./chest_xray/test/NORMAL/')
img_test_neumonia = abrirImagenesEscaladas('./chest_xray/test/PNEUMONIA/')

data = (img_train_normal, img_train_normal, img_test_normal, img_test_neumonia)


In [ ]:
def balancear_datos(imagenes_entrenamiento):

    img_train_normal = imagenes_entrenamiento[0]
    img_train_neumonia =imagenes_entrenamiento[1]
    img_test_normal =imagenes_entrenamiento[2]
    img_test_neumonia = imagenes_entrenamiento[3]

    # MAX NUMBER OF IMAGES
    n_train = min(len(img_train_normal), len(img_train_neumonia))
    n_test = min(len(img_test_normal), len(img_test_neumonia))

    # BALANCE
    img_train_normal = img_train_normal[:n_train]
    img_train_neumonia = img_train_neumonia[:n_train]
    img_test_normal = img_test_normal[:n_test]
    img_test_neumonia = img_test_neumonia[:n_test]


    # ADD LABELS
    img_train_normal = np.concatenate((img_train_normal, np.zeros((n_train,1))), axis=-1)
    img_train_neumonia = np.concatenate((img_train_neumonia, np.ones((n_train,1))), axis=-1)
    img_test_normal = np.concatenate((img_test_normal, np.zeros((n_test,1))), axis=-1)
    img_test_neumonia = np.concatenate((img_test_neumonia, np.ones((n_test,1))), axis=-1)


    # CONCATENATE
    traing_data = np.concatenate((img_train_normal, img_train_neumonia), axis=0)
    test_data = np.concatenate((img_test_normal, img_test_neumonia), axis=0)
    
    return traing_data, test_data

In [ ]:
train, test = balancear_datos(data)

# EJERCISIO 1

In [ ]:
def L(i,w,b):
    """_summary_

    Args:
        i (Vector): imagen reshaped a un vector de tamano 32^2
        w (Vector): Pesos de la red
        b (Float): Bias de la red

    Returns:
        probabolidad: 0 < p < 1: Probabilidad de que la imagen sea un 1 (Tiene neumonia)
    """
    tan = np.tanh(np.dot(w,i)+b)
    return (tan + 1)/2

In [ ]:
# Derivada de L con respecto a W
def L_w(i,w,b):
    """_summary_

    Args:
        i (Vector): imagen reshaped a un vector de tamano 32^2
        w (Vector): Pesos de la red
        b (Float): Bias de la red

    Returns:
        Vector: Gradiente de la probabilidad con respecto a los pesos
    """
    # (((b+i⊤⋅w)2+1)^(−1))/2⋅w
    return (1/((b+np.dot(i,w))**2+1))/2 * w

In [ ]:
# Derivada de L con respecto a b
def L_b(i,w,b):
    """_summary_

    Args:
        i (Vector): imagen reshaped a un vector de tamano 32^2
        w (Vector): Pesos de la red
        b (Float): Bias de la red

    Returns:
        Float: Gradiente de la probabilidad con respecto al bias
    """
    # (((b+i⊤⋅w)2+1)^(−1))/2
    return (1/((b+np.dot(i,w))**2+1))/2

# EJERCISIO 2

In [ ]:
def desenso_gradiente(i, w, b, alpha=0.1):
    """_summary_

    Args:
        imagenes_entrenamiento (List): Lista de imagenes de tamano 32^2
        w (Vector): Pesos de la red
        b (Float): Bias de la red
        alpha (Float): Learning rate

    Returns:
        Tuple: Pesos y bias actualizados

    """
    w = w - alpha*L_w(i,w,b)
    b = b - alpha*L_b(i,w,b)
    return w,b



# Ejercisio 3

In [ ]:
def split_train_test(imagenes, porcentaje=0.8):
    """_summary_

    Args:
        imagenes (List): Lista de imagenes de tamano 32^2
        porcentaje (Float): Porcentaje de imagenes que se usaran para entrenar

    Returns:
        Tuple: Listas de imagenes de entrenamiento y de test
    """
    n = len(imagenes)
    n_train = int(n*porcentaje)
    n_test = n - n_train

    imagenes_train = imagenes[:n_train]
    imagenes_test = imagenes[n_train:]

    return imagenes_train, imagenes_test

In [ ]:
def train(imagenes, w, b, alpha=0.1, epochs=100):
    """_summary_

    Args:
        imagenes_entrenamiento (List): Lista de imagenes de tamano 32^2
        w (Vector): Pesos de la red
        b (Float): Bias de la red
        alpha (Float): Learning rate
        epochs (Int): Numero de iteraciones

    Returns:
        Tuple: Pesos y bias actualizados
    """
    w = np.random.rand(32,1)
    for _ in range(epochs):
        error = 0
        for i in imagenes:
            w,b = desenso_gradiente(i,w,b,alpha)
            error += (L(i,w,b)-1)**2 # Falta arreglar esto
    return w,b